In [1]:
from tkinter import *
import tkinter as tk
import tkinter.filedialog
from tkinter.filedialog import *
import cv2
import numpy as np
import glob
from PIL import Image, ImageTk
from tkinter import filedialog
from matplotlib import pyplot as plt

In [ ]:
def LoadingImage():

        filename = filedialog.askopenfilename()
        img = cv2.imread(filename)
        cv2.imshow('image',img)
        cv2.imwrite("Saved Photos/Image.jpg", img)
        global mainImg
        mainImg = img

#Edge Detectors Operators
def Sobel():
        img = cv2.cvtColor(mainImg, cv2.COLOR_BGR2GRAY)
        sobel = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)        
        cv2.imshow('Sobel',sobel)
        cv2.imwrite("Saved Photos/SobelResults.jpg", sobel)

def Laplacian():
        img = cv2.cvtColor(mainImg, cv2.COLOR_BGR2GRAY)
        laplacian = cv2.Laplacian(img,cv2.CV_64F)    
        cv2.imshow('Laplacian',laplacian)
        cv2.imwrite("Saved Photos/LaplacianResults.jpg", laplacian)
    
def Canny ():
        img = cv2.cvtColor(mainImg, cv2.COLOR_BGR2GRAY)        
        canny = cv2.Canny(img,50,100)
        cv2.imshow('canny', canny)
        cv2.imwrite("Saved Photos/CannyResults.png", Canny)
    
 #Feature Detection
def Harris():
        img = cv2.cvtColor(mainImg, cv2.COLOR_BGR2GRAY)
        gray = np.float32(img)
        dst = cv2.cornerHarris(gray,2,3,0.04)

        #result is dilated for marking the corners, not important
        dst = cv2.dilate(dst,None)

        # Threshold for an optimal value, it may vary depending on the image.
        img[dst>0.01*dst.max()]=[255]

        cv2.imshow('Corners',img)
        cv2.imwrite("Saved Photos/HarrisResults.jpg", img)

def ORB():
    img1 = cv2.cvtColor(mainImg, cv2.COLOR_BGR2GRAY)
    filename2 = filedialog.askopenfilename()
    img2 = cv2.imread(filename2)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)      

    orb = cv2.ORB_create()

    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = bf.match(des1,des2)

    matches = sorted(matches, key = lambda x:x.distance)

    img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches[:30],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    cv2.imshow('ORB', img3)
    cv2.imwrite("Saved Photos/ORBResults.png", img3)
    
def FLANN():
    img1 = cv2.cvtColor(mainImg, cv2.COLOR_BGR2GRAY)
    filename2 = filedialog.askopenfilename()
    img2 = cv2.imread(filename2)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) 
    
    sift = cv2.xfeatures2d.SIFT_create()
# find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
# FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=20)   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
# Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]
# ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            matchesMask[i]=[1,0]
    draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv2.DrawMatchesFlags_DEFAULT)
    img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
    cv2.imshow('FLANN', img3)
    cv2.imwrite("Saved Photos/FLANNResults.png", img3)
    
def FAST():
    img = cv2.cvtColor(mainImg, cv2.COLOR_BGR2GRAY)
    fast = cv2.FastFeatureDetector_create()
    kp = fast.detect(img,None)
    img2 = cv2.drawKeypoints(img, kp, None, color=(255,0,0))

    cv2.imshow('FAST', img2)
    cv2.imwrite("Saved Photos/FASTResults.png", img2)
    
# Noise Functions
def SaltPepper():
        row,col,ch = mainImg.shape
        sNp = 0.75
        amount = 0.05
        out = np.copy(mainImg)
        
        # Salt mode
        num_salt = np.ceil(amount * mainImg.size * sNp)
        coords = [np.random.randint(0, i - 1, int(num_salt))
            for i in mainImg.shape]
        out[coords] = 1
        
        # Pepper mode
        num_pepper = np.ceil(amount* mainImg.size * (1. - sNp))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in mainImg.shape]
        out[coords] = 0
        
        cv2.imshow('Salt & Pepper',out)
        cv2.imwrite("Saved Photos/Salt&PepperResults.jpg", out)
        
def Gaussian():
        row,col,ch= mainImg.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = mainImg + gauss
        cv2.imshow('Gaussian',noisy)
        cv2.imwrite("Saved Photos/GaussianResults.jpg", noisy)
        
def Poisson():
        vals = len(np.unique(mainImg))
        vals = 2 ** np.ceil(np.log2(vals))
        poisson = np.random.poisson(mainImg * vals) / float(vals)
        cv2.imshow('Poisson',poisson)
        cv2.imwrite("Saved Photos/PoissonResults.jpg", poisson)
        
def Speckle():
        row,col,ch = mainImg.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        speck = mainImg + mainImg * gauss
        cv2.imshow('Speckle',speck)
        cv2.imwrite("Saved Photos/SpeckleResults.jpg", speck)
        
# Filtering
def GaussianBlur():
        blur = cv2.GaussianBlur(mainImg,(5,5),0)
        cv2.imshow('GaussBlur',blur)
        cv2.imwrite("Saved Photos/GaussianblurResults.jpg", blur)
        
def Medianfilter():
    median = cv2.medianBlur(mainImg,5)
    cv2.imshow('MedianBlur',median)
    cv2.imwrite("Saved Photos/MedianResults.jpg", median)
    
def Bilateralfilter():
    bilateral = cv2.bilateralFilter(mainImg,9,75,75)
    cv2.imshow('BilateralBlur',bilateral)
    cv2.imwrite("Saved Photos/BilateralResults.jpg", bilateral)
    
def Averagingfilter():
    avg = cv2.blur(mainImg,(5,5))
    cv2.imshow('Averageblur',avg)
    cv2.imwrite("Saved Photos/AverageResults.jpg", avg)
    
        
def HoughTransformCircle ():
        img = cv2.cvtColor(mainImg, cv2.COLOR_BGR2GRAY)
        img = cv2.medianBlur(img,5)
        img1 = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
     
        circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,
                                 param1=50,param2=30,minRadius=10,maxRadius=90)
    
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            # draw the outer circle
            cv2.circle(img1,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(img1,(i[0],i[1]),2,(0,0,255),3)
            
        cv2.imshow('image', img1)
        cv2.imwrite("Saved Photos/HTFcirResult.jpg", img1)
        
def HoughTransformLine ():
        img = cv2.cvtColor(mainImg,cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(img,50,400,apertureSize = 3)
        minLineLength = 300
        maxLineGap = 7
        lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
        for x1,y1,x2,y2 in lines[0]:
            cv2.line(mainImg,(x1,y1),(x2,y2),(0,255,255),3)
        cv2.imshow('image', mainImg)
        cv2.imwrite("Saved Photos/HTFLineResult.jpg", mainImg)
        
        
def FourierTransform():
        f = np.fft.fft2(mainImg)
        fshift = np.fft.fftshift(f)
        magnitude_spectrum = 20*np.log(np.abs(fshift))

        cv2.imshow('FourierTransform',magnitude_spectrum)
        cv2.imwrite("Saved Photos/FourierTransform.jpg", magnitude_spectrum)
        


In [ ]:
#GUI
root = Tk()
root.title('Computer Vision OpenCV ToolBox')
root.geometry('600x10')

menu= Menu(root)
root.config(menu=menu)


subMenu = Menu(menu)
menu.add_cascade(label="Input", menu=subMenu)
subMenu.add_command(label="Load Image",command =LoadingImage)

subMenu1 = Menu(menu)
menu.add_cascade(label="Edge Detection", menu=subMenu1)
subMenu1.add_command(label="Sobel",command =Sobel)
subMenu1.add_command(label="Laplacian",command =Laplacian)
subMenu1.add_command(label="Canny",command =Canny)

subMenu2 = Menu(menu)
menu.add_cascade(label="Feature Detection", menu=subMenu2)
subMenu2.add_command(label="Harris",command =Harris)
subMenu2.add_command(label="ORB",command =ORB)
subMenu2.add_command(label="FLANN",command =FLANN)
subMenu2.add_command(label="FAST",command =FAST)

subMenu3 = Menu(menu)
menu.add_cascade(label="Noise", menu=subMenu3)
subMenu3.add_command(label="Salt&Pepper",command =SaltPepper)
subMenu3.add_command(label="Gaussian",command =Gaussian)
subMenu3.add_command(label="Poisson",command =Poisson)
subMenu3.add_command(label="Speckle",command =Speckle)

subMenu4 = Menu(menu)
menu.add_cascade(label="Filters", menu=subMenu4)
subMenu4.add_command(label="Average Filter",command =Averagingfilter)
subMenu4.add_command(label="Gaussian Filter",command =GaussianBlur)
subMenu4.add_command(label="Median Filter",command =Medianfilter)
subMenu4.add_command(label="Bilateral Filter",command =Bilateralfilter)

subMenu5 = Menu(menu)
menu.add_cascade(label="HoughTransform", menu=subMenu5)
subMenu5.add_command(label="Extract Circle",command = HoughTransformCircle)
subMenu5.add_command(label="Extract Lines",command = HoughTransformLine)


subMenu6 = Menu(menu)
menu.add_cascade(label="FourierTransform", menu=subMenu6)
subMenu6.add_command(label="FourierTransform",command = FourierTransform)


root.mainloop()